In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../bar_movement/data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_2022-2023.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

In [5]:
def beep_boop(row):
    macdhist, ema50, mid_low, mid_high = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High']]

    if float(macdhist) > 0 and float(mid_low) > float(ema50):
        return 1

    elif float(macdhist) < 0 and float(mid_high) < float(ema50):
        return 2

    else:
        return 0
    
def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

In [6]:
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
macd = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
macdsignal = pd.Series.ewm(macd, span=9).mean()
df['macdhist'] = macd - macdsignal
df['beep_boop'] = df.apply(beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,lower_atr_band,upper_atr_band,atr,ema50,macdhist,beep_boop
0,2022-06-15 19:00:00+00:00,1.04214,1.04639,1.04137,1.04534,1.04237,1.04661,1.04160,1.04552,1.04226,1.04650,1.04149,1.04543,24613,1.034141,1.056719,0.003763,1.043661,0.000067,0
1,2022-06-15 20:00:00+00:00,1.04533,1.04690,1.04437,1.04456,1.04549,1.04708,1.04456,1.04477,1.04541,1.04698,1.04446,1.04466,5364,1.033736,1.055584,0.003641,1.043748,0.000195,1
2,2022-06-15 21:00:00+00:00,1.04394,1.04456,1.04394,1.04450,1.04494,1.04511,1.04454,1.04488,1.04444,1.04474,1.04437,1.04469,410,1.034696,1.054684,0.003331,1.043826,0.000269,1
3,2022-06-15 22:00:00+00:00,1.04439,1.04526,1.04418,1.04435,1.04498,1.04545,1.04436,1.04452,1.04468,1.04535,1.04427,1.04444,2790,1.035119,1.053761,0.003107,1.043875,0.000291,1
4,2022-06-15 23:00:00+00:00,1.04433,1.04580,1.04417,1.04567,1.04450,1.04596,1.04434,1.04584,1.04442,1.04588,1.04426,1.04576,2426,1.036529,1.054991,0.003077,1.044019,0.000359,1


In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [8]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(n_in_a_row, pips_to_risk, invert, sl_multiplier):
    reward, day_fees, n_wins, n_losses, win_streak, loss_streak, curr_win_streak, curr_loss_streak, n_buys, n_sells, n_no_trades = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    trade = None

    for i in range(n_in_a_row, len(df)):
        curr_date, curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Date', 'Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        if trade is None:
            atr, atr_lb, atr_ub = df.loc[df.index[i - 1], ['atr', 'lower_atr_band', 'upper_atr_band']]
            beep_boops = list(df.loc[df.index[i - n_in_a_row:i], 'beep_boop'])

            buy_signal = all([b_boop == 1 for b_boop in beep_boops])
            sell_signal = all([b_boop == 2 for b_boop in beep_boops])

            if invert:
                buy_signal, sell_signal = sell_signal, buy_signal

            if buy_signal:
                open_price = float(curr_ao)

                if pips_to_risk == 'atr':
                    sl_pips = atr

                elif pips_to_risk == 'atr_bands':
                    sl_pips = open_price - atr_lb 

                else:
                    sl_pips = pips_to_risk / divider

                stop_loss = round(open_price - sl_pips * sl_multiplier, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1

            elif sell_signal:
                open_price = float(curr_bo)

                if pips_to_risk == 'atr':
                    sl_pips = atr

                elif pips_to_risk == 'atr_bands':
                    sl_pips = atr_ub - open_price

                else:
                    sl_pips = pips_to_risk / divider

                stop_loss = round(open_price + sl_pips * sl_multiplier, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * 0.1:
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1

            else:
                n_no_trades += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
                win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
                loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, n_no_trades


In [9]:
# n_in_a_row, pips_to_risk, invert, sl_multiplier
n_in_a_row_vals = [3, 5, 7, 9, 11]
pips_to_risk_vals = [20, 30, 50, 'atr', 'atr_bands']
invert_vals = [True, False]
sl_multipliers = [0.5, 1.0, 1.5, 2.0]

all_combos = []

for n_in_a_row in n_in_a_row_vals:
    for pips_to_risk in pips_to_risk_vals:
        for invert in invert_vals:
            for sl_multiplier in sl_multipliers:
                all_combos.append((n_in_a_row, pips_to_risk, invert, sl_multiplier))

best_n_in_a_row, best_pips_to_risk, best_invert_val, best_sl_multiplier = None, None, None, None
top_n_results, best_rewards, best_reward, runs_finished = 10, [], -np.inf, 0

percentage_to_try = 1.0
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

for n_in_a_row, pips_to_risk, invert, sl_multiplier in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, n_no_trades = run_simulation(n_in_a_row, pips_to_risk, invert, sl_multiplier)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num no trades: ' + str(n_no_trades))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print('Min win amount: ' +  str(min(win_amounts)))
        print('Max win amount: ' + str(max(win_amounts)))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print('Min loss amount: ' +  str(min(loss_amounts)))
        print('Max loss amount: ' + str(max(loss_amounts)))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    total_profit = reward + day_fees

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or total_profit > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(total_profit), 'n_in_a_row': n_in_a_row, 'pips_to_risk': pips_to_risk, 'invert': invert, 'sl_multiplier': sl_multiplier})

    if total_profit > best_reward:
        best_reward = total_profit
        best_n_in_a_row, best_pips_to_risk, best_invert_val, best_sl_multiplier = n_in_a_row, pips_to_risk, invert, sl_multiplier
 
    print('Best reward so far: ' + str(best_reward))
    print()


Num runs: 200

-150.02000000001414 -131.59795999999997 -281.61796000001414
Num buys: 69
Num sells: 70
Num trades: 139
Num no trades: 4285
Num wins: 30
Num losses: 86
Win streak: 8
Loss streak: 13
Avg pips risked: 0.002499999999999998
Avg win amount: 138.3305833333326
Min win amount: 49.99749999999894
Max win amount: 299.99999999999807
Avg loss amount: -39.81423611111104
Min loss amount: -49.999999999998934
Max loss amount: 0.0
Remaining runs: 199
Best reward so far: -281.61796000001414

49.99514999998382 -16.90596 33.08918999998382
Num buys: 10
Num sells: 4
Num trades: 14
Num no trades: 5393
Num wins: 3
Num losses: 9
Win streak: 2
Loss streak: 9
Avg pips risked: 0.001665714285714302
Avg win amount: 166.66197999999943
Min win amount: 49.99847999999714
Max win amount: 249.99129999999943
Avg loss amount: -40.908253636364954
Min loss amount: -49.99968000000374
Max loss amount: 0.0
Remaining runs: 198
Best reward so far: 33.08918999998382

-449.9780100000024 -463.5573200000002 -913.53533000

In [10]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best best n in a row: ' + str(best_n_in_a_row))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best best invert val: ' + str(best_invert_val))
print('Best sl multiplier: ' + str(best_sl_multiplier))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 1200.5258599999913
Best best n in a row: 9
Best pips to risk: 50
Best best invert val: True
Best sl multiplier: 1.0
-----------------------
Top results:
{'reward': 799, 'n_in_a_row': 7, 'pips_to_risk': 30, 'invert': True, 'sl_multiplier': 0.5}
{'reward': 967, 'n_in_a_row': 5, 'pips_to_risk': 30, 'invert': True, 'sl_multiplier': 0.5}
{'reward': 1200, 'n_in_a_row': 9, 'pips_to_risk': 50, 'invert': True, 'sl_multiplier': 1.0}
{'reward': 897, 'n_in_a_row': 3, 'pips_to_risk': 'atr_bands', 'invert': True, 'sl_multiplier': 1.0}
{'reward': 922, 'n_in_a_row': 9, 'pips_to_risk': 30, 'invert': True, 'sl_multiplier': 2.0}
{'reward': 1156, 'n_in_a_row': 11, 'pips_to_risk': 'atr_bands', 'invert': True, 'sl_multiplier': 1.0}
{'reward': 892, 'n_in_a_row': 9, 'pips_to_risk': 'atr_bands', 'invert': True, 'sl_multiplier': 0.5}
{'reward': 644, 'n_in_a_row': 3, 'pips_to_risk': 30, 'invert': True, 'sl_multiplier': 1.5}
{'reward': 1061, 'n_in_a_row': 7, 'p